# Computing daily ONN index on Binance

## Getting all available symbols on Binance

In this section, we just get all available assets on Binance, generating at the end a file named `all-binance-tokens.csv`. Each line in this file contains the asset symbol two times, separated by comma (e.g. `ETH,ETH`). This is because we will use it afterwards in the __[`bnbFetchCoinData.py`](https://github.com/srmq/bnbfetch-coin-data)__ script, which expects the CSV file of lines in the format `asset_description,asset_symbol`, and we don't care about descriptions here.

In [18]:
import httpx
import pandas as pd

In [2]:
binanceBaseURL = 'https://api.binance.com'

In [3]:
async def getExchangeInfo():
    spotExchangeInfoURL = binanceBaseURL + "/api/v3/exchangeInfo"

    async with httpx.AsyncClient() as client:
        r = await client.get(spotExchangeInfoURL)
    return r.json()

In [4]:
exchangeInfo = await getExchangeInfo()

In [5]:
#blacklist = None
#with open('blacklist.txt', 'r') as f:
#    blacklist = f.read().split(',')
# blacklist should be disabled for now, we need coin data for every coin

In [6]:
baseAssets = set()
#for coinSymbol in exchangeInfo['symbols']:
#    if not (coinSymbol['baseAsset'].lower() in blacklist):
#        baseAssets.add(coinSymbol['baseAsset'])

In [7]:
for coinSymbol in exchangeInfo['symbols']:
    baseAssets.add(coinSymbol['baseAsset'])

In [8]:
def notLeveraged(asset):
    return not (asset.endswith('UP') or asset.endswith('DOWN') or asset.endswith('BEAR') or asset.endswith('BULL'))

In [9]:
baseAssets = set(filter(notLeveraged, baseAssets))

In [10]:
baseAssets

{'1INCH',
 'AAVE',
 'ACA',
 'ACH',
 'ACM',
 'ADA',
 'ADX',
 'AE',
 'AERGO',
 'AGI',
 'AGIX',
 'AGLD',
 'AION',
 'AKRO',
 'ALCX',
 'ALGO',
 'ALICE',
 'ALPACA',
 'ALPHA',
 'ALPINE',
 'AMB',
 'AMP',
 'ANC',
 'ANKR',
 'ANT',
 'ANY',
 'APE',
 'API3',
 'APPC',
 'AR',
 'ARDR',
 'ARK',
 'ARN',
 'ARPA',
 'ASR',
 'AST',
 'ASTR',
 'ATA',
 'ATM',
 'ATOM',
 'AUCTION',
 'AUD',
 'AUDIO',
 'AUTO',
 'AVA',
 'AVAX',
 'AXS',
 'BADGER',
 'BAKE',
 'BAL',
 'BAND',
 'BAR',
 'BAT',
 'BCC',
 'BCD',
 'BCH',
 'BCHA',
 'BCHABC',
 'BCHSV',
 'BCN',
 'BCPT',
 'BDOT',
 'BEAM',
 'BEL',
 'BETA',
 'BETH',
 'BGBP',
 'BICO',
 'BIFI',
 'BKRW',
 'BLZ',
 'BNB',
 'BNT',
 'BNX',
 'BOND',
 'BOT',
 'BQX',
 'BRD',
 'BSW',
 'BTC',
 'BTCB',
 'BTCST',
 'BTG',
 'BTS',
 'BTT',
 'BTTC',
 'BURGER',
 'BUSD',
 'BZRX',
 'C98',
 'CAKE',
 'CDT',
 'CELO',
 'CELR',
 'CFX',
 'CHAT',
 'CHESS',
 'CHR',
 'CHZ',
 'CITY',
 'CKB',
 'CLOAK',
 'CLV',
 'CMT',
 'CND',
 'COCOS',
 'COMP',
 'COS',
 'COTI',
 'COVER',
 'CREAM',
 'CRV',
 'CTK',
 'CTSI',
 'CTXC

In [11]:
with open('all-binance-tokens.csv', 'w') as f:
    for asset in baseAssets:
        f.write(f"{asset},{asset}\n")

Now that we have our `all-binance-tokens.csv` file, we can download, for instance, the complete historical data for all assets on Binance, using something like:

`python <path_to_script>/bnbFetchCoinData.py --coins all-binance-tokens.csv --interval 1d`

This will create a `coindata` directory under the current one with everything we need.

## Importing Data to QuestDB

We will use __[QuestDB](https://questdb.io/)__ in order to facilitate data manipulation and keep memory requirements low (we won't need to load everything on RAM). First we will create a table for holding the data. Here is what we did:

`CREATE TABLE binance_daily(symb SYMBOL capacity 2048, open_time TIMESTAMP, high DOUBLE, low DOUBLE, close DOUBLE, volume DOUBLE, close_time TIMESTAMP, quote_asset_vol DOUBLE, num_trades LONG, takerbuy_baseasset_vol DOUBLE, takerbuy_quoteasset_vol DOUBLE), index(symb) timestamp(open_time) PARTITION BY YEAR;`

Now we will import the data (that went to the `coindata` subdirectory)...

In [1]:
import glob

allCSVFiles = glob.glob('./coindata/*.csv')

In [2]:
import httpx
import csv

host = 'http://localhost:9000'

async def questInsert(symbol: str, filename: str, lotSize=300):
    async def executeQuery(valStrings):
        sql_query = "INSERT INTO binance_daily VALUES"
        sql_query += valStrings[0]
        for i in range(1, len(valStrings)):
            sql_query += f",{valStrings[i]}"
        query_params = {'query': sql_query, 'fmt': 'json'}
        async with httpx.AsyncClient() as client:
            await client.get(host + '/exec', params=query_params)
        
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        valStrings = []
        for row in reader:
            if len(valStrings) == lotSize:
                await executeQuery(valStrings)
                valStrings = []
            val = f"(\'{symbol}\', {int(row['Open time'])*1000}, {float(row['High'])}, {float(row['Low'])}, {float(row['Close'])}, {float(row['Volume'])}, {int(row['Close time'] + '999')}, {float(row['Quote asset volume'])}, {int(row['Number of trades'])}, {float(row['Taker buy base asset volume'])}, {float(row['Taker buy quote asset volume'])})"
            valStrings.append(val)
        if len(valStrings) > 0:
            await executeQuery(valStrings)

In [3]:
from pathlib import Path
i = 0
for csvFile in allCSVFiles:
    symb = Path(csvFile).name.split('.')[0]
    await questInsert(symb, csvFile)
    i += 1
    if i % 100 == 0:
        print(f"Inserted {i} symbols out of {len(allCSVFiles)}")    

Inserted 100 symbols out of 1819
Inserted 200 symbols out of 1819
Inserted 300 symbols out of 1819
Inserted 400 symbols out of 1819
Inserted 500 symbols out of 1819
Inserted 600 symbols out of 1819
Inserted 700 symbols out of 1819
Inserted 800 symbols out of 1819
Inserted 900 symbols out of 1819
Inserted 1000 symbols out of 1819
Inserted 1100 symbols out of 1819
Inserted 1200 symbols out of 1819
Inserted 1300 symbols out of 1819
Inserted 1400 symbols out of 1819
Inserted 1500 symbols out of 1819
Inserted 1600 symbols out of 1819
Inserted 1700 symbols out of 1819
Inserted 1800 symbols out of 1819
